In [3]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import numpy as np
import pandas as pd
import sys
sys.path.append("..")

In [4]:
# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
def one_hot(seq):
    conversion_dict={
        'A':np.array([1.0,0.0,0.0,0.0]),
        'C':np.array([0.0,1.0,0.0,0.0]),
        'G':np.array([0.0,0.0,1.0,0.0]),
        'T':np.array([0.0,0.0,0.0,1.0])
    }
    seqlen = len(seq)
    enc_arr=conversion_dict[seq[0]]
    for i in seq[1:]:
        enc_arr=np.vstack((enc_arr,conversion_dict[i]))
    enc_arr=enc_arr.T.reshape((1,4,50))
    
    return torch.tensor(enc_arr, dtype = torch.float32)
        
    

In [98]:
testseq="AGACTAGCAGCATGCATCAGTCGCTAGCATCGATCGGATCGATGTCGTAG"
print(len(testseq))
testtensor=one_hot(testseq)
print(testtensor.shape)

50
torch.Size([1, 4, 50])


In [71]:

# load data
dataset1=pd.read_csv('data/random_train_pc.csv')

class fiveprimeutrs(Dataset):
    def __init__(self, seqs, mrl):
        self.seqs = seqs
        self.mrl = mrl
    def __len__(self):
        return len(self.seqs)
    def __getitem__(self, idx):
        seq = self.seqs[idx]
        mrl = self.mrl[idx]
        

In [10]:
#Model
class Model(nn.Module):

    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv1d(4, 120, kernel_size = 8, padding='same')
        self.conv2 = nn.Conv1d(120, 120, kernel_size = 8, padding='same')
        self.conv3 = nn.Conv1d(120, 120, kernel_size = 8, padding='same')
        self.flat1 = nn.Flatten()
        self.fc1 = nn.Linear(6000, 40)
        self.drop1 = nn.Dropout(0.2)
        self.out = nn.Linear(40, 1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        print(x.shape)
        x = F.relu(self.conv2(x))
        print(x.shape)
        x = F.relu(self.conv3(x))
        print(x.shape)
        x = self.flat1(x)
        #x= torch.transpose(x, 1, 2)
        x = F.relu(self.fc1(x))
        x = self.drop1(x)
        x = self.out(x)
        return x



In [11]:
net=Model()
net.to(device)

Model(
  (conv1): Conv1d(4, 120, kernel_size=(8,), stride=(1,), padding=same)
  (conv2): Conv1d(120, 120, kernel_size=(8,), stride=(1,), padding=same)
  (conv3): Conv1d(120, 120, kernel_size=(8,), stride=(1,), padding=same)
  (flat1): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=6000, out_features=40, bias=True)
  (drop1): Dropout(p=0.2, inplace=False)
  (out): Linear(in_features=40, out_features=1, bias=True)
)

In [13]:
testseq="AGACTAGCAGCATGCATCAGTCGCTAGCATCGATCGGATCGATGTCGTAG"
testtensor=one_hot(testseq)
train_dat=torch.utils.data.TensorDataset(testtensor.to(device))
train_loader=torch.utils.data.DataLoader(train_dat, batch_size=1, shuffle=False)
test_loader=torch.utils.data.DataLoader(testtensor, batch_size=1, shuffle=False)
print(testtensor.shape)
net(testtensor)
# zum error: um kernel size 8,4 zu verwnden muss es eine 2d convolutional layer sein und nicht 1d, ich muss nur den input reshapen


torch.Size([1, 4, 50])
torch.Size([1, 120, 50])
torch.Size([1, 120, 50])
torch.Size([1, 120, 50])


tensor([[-0.0549]], grad_fn=<AddmmBackward0>)